In [4]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from dotenv import dotenv_values

config = dotenv_values(".env")

# ---- Define State ----
class ChatState(TypedDict):
    messages: Annotated[list, add_messages]  # conversation history (auto-appends)
    user_name: str                           # extra variable
    turn_count: int                          # tracks number of turns

# ---- Model ----
llm = ChatOpenAI(api_key=config["OPEN_AI_API_KEY"], model="gpt-4o-mini", temperature=0.1)

# ---- Nodes ----
def greet_user(state: ChatState):
    """First node: greet user using stored variable"""
    reply = f"Hello {state['user_name']} 👋! How can I help you today?"
    return {"messages": [{"role": "assistant", "content": reply}]}


def chat_node(state: ChatState):
    """Main chat node: uses both messages + other variables"""
    # update turn count
    new_count = state["turn_count"] + 1
    
    # generate response
    response = llm.invoke(state["messages"])
    
    return {
        "messages": [response],   # auto-appended by add_messages
        "turn_count": new_count
    }

# ---- Graph ----
workflow = StateGraph(ChatState)

workflow.add_node("greet", greet_user)
workflow.add_node("chat", chat_node)

workflow.set_entry_point("greet")
workflow.add_edge("greet", "chat")
workflow.add_edge("chat", END)

graph = workflow.compile()

# ---- Run Example ----
input_state = {
    "messages": [{"role": "user", "content": "Hi!"}],
    "user_name": "Anurup",
    "turn_count": 0,
}

final_state = graph.invoke(input_state)

print("Final Messages:")
for m in final_state["messages"]:
    print(m)

print("\nTurn count:", final_state["turn_count"])

Final Messages:
content='Hi!' additional_kwargs={} response_metadata={} id='a811e6e7-a242-4e46-ba03-3f01a757e8f9'
content='Hello Anurup 👋! How can I help you today?' additional_kwargs={} response_metadata={} id='caf2b0fb-5f4a-4a45-b2cf-6cdf52aa4fcd'
content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 27, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C8qCanxYSLzJ9kV34gaUpOADPyRAi', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--e3643d91-e4af-4662-b292-c2a892c2f38b-0' usage_metadata={'input_tokens': 27, 'output_tokens': 9, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cach